Cell for working not locally

In [3]:
# !git clone https://github.com/RodionfromHSE/YAGPT.git
# !cd YAGPT
# !git checkout dev

In [1]:
import json
from pprint import pprint

params = json.load(open('config.json'))
onServer = True
if not onServer:
    params['num_proc'] = 1
pprint(params)

{'additional_special_tokens': ['history: ', 'speaker1: ', 'speaker2: '],
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'dataset_name': 'under-tree/prepared-yagpt',
 'num_proc': 8}


In [2]:
%%capture
!pip install transformers datasets huggingface_hub

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from datasets import load_dataset
from pprint import pprint

name = params['dataset_name']
dataset = load_dataset(name)
pprint(dataset.info)

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer


checkpoint = params['checkpoint']
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': params['additional_special_tokens']}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))
None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def tokenize(elem):
    inputs = tokenizer(elem['text'])
    return inputs

data_token = dataset.map(prepare_text, batched=True, remove_columns=dataset.column_names)

In [ ]:
block_size = params['block_size']

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

data_prep = dataset.map(group_texts, batched=True, num_proc=params['num_proc'])

In [1]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch' # evaluation strategy to adopt during training
    
)

trainer = Trainer(
    model=model,                         
    args=training_args,
    train_dataset=data_prep['train'],
    eval_dataset=data_prep['test']
)

ModuleNotFoundError: ignored

In [21]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
import math

eval_result = trainer.evaluate() # returns a dict with the evaluation metrics, cross-entropy loss and perplexity
pprint(eval_result)